In [1]:
from datetime import datetime

In [2]:
datetime.today()

datetime.datetime(2019, 5, 1, 17, 15, 12, 956432)

In [8]:
currentYear = datetime.today().year
currentYear

2019

In [3]:
currentMonth = datetime.today().month
currentDate = datetime.today().day

currentYear = str(currentYear)[-2:]

if (currentMonth < 10):
    currentMonth = "0" + str(currentMonth)
else:
    currentMonth = str(currentMonth)
    
if (currentDate < 10):
    currentDate = "0" + str(currentDate)
else:
    currentDate = str(currentDate)


today = currentYear + "-" + currentMonth + "-" + currentDate
# print(currentYear + "-" + currentMonth + "-" + currentDate)
print(today)

19-05-01


In [4]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:



# coding: utf-8

# In[8]:


import os
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
from pandas.io.json import json_normalize
import tabula 
from tabula import wrapper
import httplib2
import os
import requests, bs4, pandas as pd, numpy as np
from datetime import datetime




def makeURL(myUrl, myKey, myParameter):
    # myUrl = "http://192.168.1.120/index.php?"
    url = myUrl + myKey + "&" + myParameter

    url = url.rstrip('&')
    return url

def xmlProcess(url):
    response = requests.get(url)
    # Check if page is up
    result = []
    if response.status_code == 200:
        # Convert webpage to %Data
        Data = BeautifulSoup(response.text, 'lxml-xml')
        rows = 0
        columnName = []
        # search Item all item tag
        iterData = Data.find_all('item')
        for item in iterData:
            item_list = []
            # Fill the value in one row
            for tag in item.find_all():
                try:
                    tagname = tag.name
                    if rows == 0:
                        columnName.append(tagname)
                    item_list.append(item.find(tagname).text)
                except Exception as e:
                    print("This row will be ignored. ", item_list)
            rows = rows + 1
            result.append(item_list)
    finalResult = pd.DataFrame(result)
    finalResult.columns = columnName
    print(finalResult)
    return finalResult


def jsonProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)
    # JSON 데이터 획득
    json = response.json()
    # PandasDataframe변환
    df = json_normalize(json)
    return df

def csvProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)
    df = pd.read_csv(url, encoding="ms949")
    return df


def pdfProcess(inputFolder, url):
    #다른 사이트에서 pdf 데이터를 받을 때는 함수를 다시 만들어야 해요
    #response = requests.get("http://fsc.go.kr/info/trd_list.jsp?menu=7230000&bbsid=BBS0069")
    resp = requests.get("http://fsc.go.kr/info/trd_list.jsp?menu=7230000&bbsid=BBS0069")
    #response = requests.get("http://fsc.go.kr/info/trd_list.jsp?menu=7230000&bbsid=BBS0069")
    resp.encoding='utf-8'
    html = resp.text
    bs = bs4.BeautifulSoup(html, 'html.parser')
   
    #데이터 추출하기
    originalData  = bs.select("#contents > div.board > table > tbody > tr > td > a")
    convertedData = str(originalData[2])
    dailyUpdated = convertedData.split('"')[1].split("amp;")[1]

    url = url + "&" + dailyUpdated # & 를 안 써줘서 직업이 안됐었어....
    df2 = wrapper.read_pdf(url,
              multiple_tables=True,
              pages="all",
              pandas_options={"header":0})
    #파일이름에 반영할 현재 날짜 구하기 
    currentYear = datetime.today().year
    currentMonth = datetime.today().month
    currentDate = datetime.today().day

    currentYear = str(currentYear)[-2:]
    if (currentMonth < 10):
        currentMonth = "0" + str(currentMonth)
    else:
        currentMonth = str(currentMonth)

    if (currentDate < 10):
        currentDate = "0" + str(currentDate)
    else:
        currentDate = str(currentDate)


    today = currentYear + currentMonth + currentDate
    
    #각각의 표를 파일로 저장
    for i in range(0, len(df2)):
        fileName = inputFolder + "_" + today + "_" + str(i) + '.csv'
        df2[i].to_csv("../../data/outbound/" + inputFolder + "/" + fileName, index=False, encoding = "ms949")
        
    #continue
    
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [5]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:



# coding: utf-8

# In[8]:


import os
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
from pandas.io.json import json_normalize
import tabula 
from tabula import wrapper
import httplib2
import os




def makeURL(myUrl, myKey, myParameter):
    # myUrl = "http://192.168.1.120/index.php?"
    url = myUrl + myKey + "&" + myParameter

    url = url.rstrip('&')
    return url

def xmlProcess(url):
    response = requests.get(url)
    # Check if page is up
    result = []
    if response.status_code == 200:
        # Convert webpage to %Data
        Data = BeautifulSoup(response.text, 'lxml-xml')
        rows = 0
        columnName = []
        # search Item all item tag
        iterData = Data.find_all('item')
        for item in iterData:
            item_list = []
            # Fill the value in one row
            for tag in item.find_all():
                try:
                    tagname = tag.name
                    if rows == 0:
                        columnName.append(tagname)
                    item_list.append(item.find(tagname).text)
                except Exception as e:
                    print("This row will be ignored. ", item_list)
            rows = rows + 1
            result.append(item_list)
    finalResult = pd.DataFrame(result)
    finalResult.columns = columnName
    print(finalResult)
    return finalResult


def jsonProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)
    # JSON 데이터 획득
    json = response.json()
    # PandasDataframe변환
    df = json_normalize(json)
    return df

def csvProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)
    df = pd.read_csv(url, encoding="ms949")
    return df


def pdfProcess(inputFolder, url):
    #다른 사이트에서 pdf 데이터를 받을 때는 함수를 다시 만들어야 해요
    #response = requests.get("http://fsc.go.kr/info/trd_list.jsp?menu=7230000&bbsid=BBS0069")
    response = requests.get(url)
    response.encoding='utf-8'
    html = response.text
    originalData  = bs.select("#contents > div.board > table > tbody > tr > td > a")
    convertedData = str(originalData[2])
    dailyUpdated = convertedData.split('"')[1].split("amp;")[1]

    url = "http://fsc.go.kr" + dailyUpdated
    bs = bs4.BeautifulSoup(html, 'html.parser')
    df2 = wrapper.read_pdf(url,
              multiple_tables=True,
              pages="all",
              pandas_options={"header":0})
    #각각의 표를 파일로 저장
    for i in range(0, len(df2)):
        fileName = inputFolder + str(i) + '.csv'
        df2[i].to_csv("../../data/outbound/" + inputFolder + "/" + fileName, index=False, encoding = "ms949")
        
    #continue
    
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [6]:
import os
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
from pandas.io.json import json_normalize
import tabula
from tabula import wrapper


def makeURL(myUrl, myKey, myParameter):
    # myUrl = "http://192.168.1.120/index.php?"
    url = myUrl + myKey + "&" + myParameter

    url = url.rstrip('&')
    return url

def xmlProcess(url):
    response = requests.get(url)
    # Check if page is up
    result = []
    if response.status_code == 200:
        # Convert webpage to %Data
        Data = BeautifulSoup(response.text, 'lxml-xml')
        rows = 0
        columnName = []
        # search Item all item tag
        iterData = Data.find_all('item')
        for item in iterData:
            item_list = []
            # Fill the value in one row
            for tag in item.find_all():
                try:
                    tagname = tag.name
                    if rows == 0:
                        columnName.append(tagname)
                    item_list.append(item.find(tagname).text)
                except Exception as e:
                    print("This row will be ignored. ", item_list)
            rows = rows + 1
            result.append(item_list)
    finalResult = pd.DataFrame(result)
    finalResult.columns = columnName
    print(finalResult)
    return finalResult


def jsonProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)
    # JSON 데이터 획득
    json = response.json()
    # PandasDataframe변환
    df = json_normalize(json)
    return df

def csvProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)

    df = pd.read_csv(url, encoding="ms949")
    return df


def pdfProcess(url):
    response = requests.get(url)
    df2 = wrapper.read_pdf(url,
              multiple_tables=True,
              pages="all",
              pandas_options={"header":0})
    #각각의 표를 파일로 저장
    for i in range(0, len(df2)):
        fileName = '금융시장동향' + str(i) + '.csv'
        df = df2[i].to_csv("../../data/outbound/" + fileName, index=False, encoding = "ms949")
        
        return df 
    
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


In [7]:

# coding: utf-8

# In[8]:


import os
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
from pandas.io.json import json_normalize
import tabula 
from tabula import wrapper

def makeURL(myUrl, myKey, myParameter):
    # myUrl = "http://192.168.1.120/index.php?"
    url = myUrl + myKey + "&" + myParameter

    url = url.rstrip('&')
    return url

def xmlProcess(url):
    response = requests.get(url)
    # Check if page is up
    result = []
    if response.status_code == 200:
        # Convert webpage to %Data
        Data = BeautifulSoup(response.text, 'lxml-xml')
        rows = 0
        columnName = []
        # search Item all item tag
        iterData = Data.find_all('item')
        for item in iterData:
            item_list = []
            # Fill the value in one row
            for tag in item.find_all():
                try:
                    tagname = tag.name
                    if rows == 0:
                        columnName.append(tagname)
                    item_list.append(item.find(tagname).text)
                except Exception as e:
                    print("This row will be ignored. ", item_list)
            rows = rows + 1
            result.append(item_list)
    finalResult = pd.DataFrame(result)
    finalResult.columns = columnName
    print(finalResult)
    return finalResult


def jsonProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)
    # JSON 데이터 획득
    json = response.json()
    # PandasDataframe변환
    df = json_normalize(json)
    return df

def csvProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)

    df = pd.read_csv(url, encoding="ms949")
    return df


def pdfProcess(url):
    response = requests.get(url)
    df2 = wrapper.read_pdf(url,
              multiple_tables=True,
              pages="all",
              pandas_options={"header":0})
    #각각의 표를 파일로 저장
    for i in range(0, len(df2)):
        fileName = '금융시장동향' + str(i) + '.csv'
        df2[i].to_csv("../../data/outbound/" +  inputFolder + "/" + fileName, index=False, encoding = "ms949")
    
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
